In [0]:
import pathlib
import pandas as pd
import json
import re
from tqdm import tqdm
import numpy as np 

In [0]:
df_pan2 = pd.read_csv("./pan_v2.csv").drop( ['Unnamed: 0','Set','Categorie Montage','Dossier Patient','Session','File N°','exam_duration','Pan_vs_SWT','Pan_vs_XQRS','interval_index','interval_start_time','label'],axis=1)
df_swt2 = pd.read_csv("./swt_v2.csv").drop(['Unnamed: 0','Set','Categorie Montage','Dossier Patient','Session','File N°','exam_duration','Pan_vs_SWT','SWT_vs_XQRS','interval_index','interval_start_time','label'], axis=1)
df_xqrs2 = pd.read_csv("./xqrs_v2.csv").drop(['Unnamed: 0','Set','Categorie Montage','Dossier Patient','Session','File N°','exam_duration','SWT_vs_XQRS','Pan_vs_XQRS','interval_index','interval_start_time','label'], axis=1)

In [0]:
df_swt2

,Patient,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1
0,8643,NaN,NaN,NaN,0.0,0.000000,3.0,17.647059,NaN,NaN,66.406250,NaN,NaN,NaN,120.000000,105.931034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8643,NaN,NaN,NaN,4.0,22.222222,6.0,33.333333,NaN,NaN,175.781250,NaN,NaN,NaN,123.870968,90.887574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8643,600.043403,48.993541,33.789686,2.0,11.111111,4.0,22.222222,34.957824,599.609375,152.343750,0.058259,0.081650,100.606071,112.941176,87.771429,7.749811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8643,636.827257,25.195199,18.188169,0.0,0.000000,5.0,27.777778,18.431833,634.765625,89.843750,0.028943,0.039564,94.356598,102.400000,88.786127,3.632898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8643,552.191840,57.491179,26.590452,2.0,11.111111,6.0,33.333333,28.287609,558.593750,189.453125,0.051228,0.104115,109.903330,140.917431,97.523810,12.377808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305517,4671,525.263158,1.910268,1.333333,0.0,0.000000,0.0,0.000000,1.333333,524.000000,4.000000,0.002538,0.003637,114.229884,114.503817,113.636364,0.403218,12.137080,1.052243,8.819451,11.534489,8.418049,2.448291,409.225628,0.378372,1.443710,12.153221,8.418049
305518,4671,523.368421,1.498537,1.617802,0.0,0.000000,0.0,0.000000,1.632993,524.000000,4.000000,0.003120,0.002863,114.642890,115.384615,114.503817,0.321176,9.553155,1.048043,25.766329,9.115229,8.268186,2.375839,366.467014,0.279132,1.340153,11.080635,8.268186
305519,4671,527.157895,1.675416,1.333333,0.0,0.000000,0.0,0.000000,1.333333,528.000000,4.000000,0.002529,0.003178,113.818985,114.503817,113.636364,0.353645,5.413311,1.112166,8.846159,4.867361,9.094540,2.360261,415.242356,0.232321,1.255105,11.414606,9.094540
305520,4671,554.888889,11.297521,2.764080,0.0,0.000000,0.0,0.000000,3.217599,560.000000,32.000000,0.005799,0.020360,108.172494,111.940299,105.633803,2.160180,9.528094,1.131946,29.942119,8.417448,6.377635,2.091061,178.862793,0.216104,1.099362,7.011329,6.377635


In [0]:
df_pan2

,Patient,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1
0,8643,NaN,NaN,NaN,1.0,3.571429,4.0,14.285714,NaN,NaN,218.750000,NaN,NaN,NaN,178.604651,108.169014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8643,NaN,NaN,NaN,7.0,25.925926,11.0,40.740741,NaN,NaN,343.750000,NaN,NaN,NaN,182.857143,89.302326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8643,NaN,NaN,NaN,3.0,10.714286,6.0,21.428571,NaN,NaN,351.562500,NaN,NaN,NaN,178.604651,87.272727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8643,NaN,NaN,NaN,5.0,17.857143,8.0,28.571429,NaN,NaN,282.226562,NaN,NaN,NaN,180.705882,97.678855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8643,NaN,NaN,NaN,6.0,26.086957,9.0,39.130435,NaN,NaN,563.151042,NaN,NaN,NaN,134.148472,59.381443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265076,4671,525.263158,1.910268,1.333333,0.0,0.000000,0.0,0.000000,1.333333,524.0,4.000000,0.002538,0.003637,114.229884,114.503817,113.636364,0.403218,11.738980,0.958917,9.259370,12.241918,8.558139,2.424336,408.072038,0.474028,1.392894,11.920583,8.558139
265077,4671,523.789474,0.917663,1.333333,0.0,0.000000,0.0,0.000000,1.333333,524.0,4.000000,0.002546,0.001752,114.550175,115.384615,114.503817,0.196680,9.419667,1.032932,25.301347,9.119348,8.554954,2.371723,383.873480,0.444686,1.311272,11.217870,8.554954
265078,4671,528.210526,2.485043,1.885618,0.0,0.000000,0.0,0.000000,1.885618,528.0,8.000000,0.003570,0.004705,113.593454,114.503817,112.781955,0.520248,5.074024,0.774316,8.832226,6.552910,9.610095,2.342531,441.933784,0.296394,1.196305,11.496603,9.610095
265079,4671,554.888889,11.297521,2.764080,0.0,0.000000,0.0,0.000000,3.217599,560.0,32.000000,0.005799,0.020360,108.172494,111.940299,105.633803,2.160180,9.045359,1.258486,29.252824,7.187493,6.110245,2.120980,173.611223,0.324004,1.162520,7.103284,6.110245


In [0]:
df_xqrs2

,Patient,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1
0,8643,NaN,NaN,NaN,5.0,26.315789,6.0,31.578947,NaN,NaN,179.687500,NaN,NaN,NaN,132.987013,95.108359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8643,537.109375,56.359305,41.722014,4.0,21.052632,4.0,21.052632,41.722578,546.875000,189.453125,0.077680,0.104931,113.041243,141.566820,97.834395,1.313485e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8643,602.213542,51.251884,26.831113,4.0,22.222222,4.0,22.222222,27.702479,585.937500,125.000000,0.046001,0.085106,100.296032,108.936170,88.786127,8.031925e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8643,564.453125,33.125314,21.511084,1.0,5.555556,2.0,11.111111,21.687087,548.828125,89.843750,0.038421,0.058686,106.617186,110.503597,94.814815,5.605881e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8643,NaN,NaN,NaN,2.0,11.111111,2.0,11.111111,NaN,NaN,125.000000,NaN,NaN,NaN,139.636364,108.169014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308940,4671,525.473684,1.982379,0.916246,0.0,0.000000,0.0,0.000000,0.942809,524.000000,4.000000,0.001794,0.003773,114.184229,114.503817,113.636364,4.184389e-01,11.091962,1.062998,8.230032,10.434598,8.199485,2.436866,388.251432,0.433293,1.443710,11.837678,8.199485
308941,4671,524.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,524.000000,0.000000,0.000000,0.000000,114.503817,114.503817,114.503817,1.421085e-14,9.152304,1.224314,24.660710,7.475457,8.156070,2.369910,356.595885,0.354601,1.340153,10.930383,8.156070
308942,4671,528.000000,2.981424,1.885618,0.0,0.000000,0.0,0.000000,1.885618,528.000000,8.000000,0.003571,0.005647,113.639796,114.503817,112.781955,6.245934e-01,4.562690,0.861958,8.318826,5.293405,9.782922,2.304168,434.296810,0.187326,1.134461,11.098341,9.782922
308943,4671,555.111111,11.023445,3.049760,0.0,0.000000,0.0,0.000000,3.360672,560.000000,28.000000,0.006054,0.019858,108.127022,111.111111,105.633803,2.101548e+00,9.082972,1.413336,28.909886,6.426618,6.653388,2.054814,182.797907,0.211488,1.032347,6.868603,6.653388


In [0]:
list_features_non_linear = [
'csi',
'cvi',
'Modified_csi',
'sampen',
'sd1',
'sd2',
'ratio_sd2_sd1',
'label']

list_features_freq=[
'lf',
'hf',
'vlf',
'lf_hf_ratio']

list_features_temp=[
'mean_nni',
'sdnn',
'sdsd',
'nni_50',
'pnni_50',
'nni_20',
'pnni_20',
'rmssd',
'median_nni',
'range_nni',
'cvsd',
'cvnni',
'mean_hr',
'max_hr',
'min_hr',
'std_hr']

list_features = list_features_non_linear+list_features_freq+list_features_temp

list_features_control =[
    'mean_nni',
    'csi',
    'hf'
]

In [0]:
df_pan2_patient = df_pan2.groupby(['Patient'],as_index=False).mean()
df_swt2_patient =  df_swt2.groupby(['Patient'],as_index=False).mean()
df_xqrs2_patient = df_xqrs2.groupby(['Patient'],as_index=False).mean()

In [0]:
df_xqrs2_patient

,Patient,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1
0,2,689.094620,20.214871,13.630261,0.664495,4.748278,1.022801,7.080544,14.052637,687.756272,60.274510,0.020464,0.029706,inf,inf,inf,2.711351,543.083750,150.288790,298.929860,4.932077,4.037827,3.509169,686.517017,inf,11.423463,39.825812,4.037827
1,17,707.162115,13.878511,6.989932,0.096774,0.762609,0.266129,2.368312,7.589630,711.121951,38.322581,0.010047,0.018255,87.069804,90.450279,85.426638,1.618941,122.302838,51.284593,114.507835,4.771547,6.103743,3.338721,1335.665529,inf,6.784225,42.215218,6.103743
2,21,696.028690,22.806232,13.164502,0.250000,1.613923,1.528846,10.459025,13.652229,696.784314,69.485437,0.019812,0.033237,87.052341,93.290598,83.619691,2.895940,382.529254,97.952686,211.854759,4.214278,4.303563,3.822175,763.514127,inf,10.523340,43.435157,4.303563
3,32,993.491279,67.479057,46.719408,2.146341,21.301321,3.634146,35.765443,48.976662,996.224490,185.178862,0.050820,0.071014,62.081716,70.224919,57.023263,4.638969,3538.150468,464.979305,1832.277570,8.364500,3.676932,4.830773,2046.067838,inf,35.572919,130.733513,3.676932
4,54,1135.040404,14.443546,10.517722,0.061538,0.683761,0.546154,6.176768,11.282227,1136.692308,40.061538,0.009967,0.012792,52.979610,54.198363,52.266691,0.650681,264.579598,14.727422,395.944448,12.751943,4.111332,3.427560,597.404172,0.863294,7.102295,30.564006,4.111332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,13182,452.621128,107.964996,78.912937,2.727273,19.283804,3.636364,25.282022,81.000349,456.805556,257.469697,0.120777,0.161595,inf,inf,inf,16.555437,0.000000,0.000000,0.000000,NaN,NaN,-inf,NaN,inf,0.000000,0.000000,NaN
599,13336,804.533667,30.412083,19.560937,0.514970,4.105575,2.688623,22.155210,20.294909,804.906627,84.165669,0.025245,0.038030,inf,inf,inf,2.799463,812.139345,183.612274,479.894534,8.484760,4.144689,4.104616,1110.554204,inf,14.950376,60.681075,4.144689
600,13345,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,inf,inf,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,13400,571.832390,34.707421,19.708418,1.427273,7.461393,2.596970,13.631439,20.297845,576.575212,121.932859,0.036677,0.063196,inf,inf,inf,7.398015,763.762012,260.933791,443.197619,6.732947,5.803596,4.044991,1587.535866,inf,15.507868,71.137202,5.803596


In [0]:
df_pan2_patient.to_csv('./pan2_patient.csv')

In [0]:
df_swt2_patient.to_csv('./swt2_patient.csv')

In [0]:
df_xqrs2_patient.to_csv('./xqrs2_patient.csv')

In [0]:
df_pan2_patientMSTD = df_pan2.groupby(['Patient'],as_index=False).agg([np.mean,np.std])
df_swt2_patientMSTD =  df_swt2.groupby(['Patient'],as_index=False).agg([np.mean,np.std])
df_xqrs2_patientMSTD = df_xqrs2.groupby(['Patient'],as_index=False).agg([np.mean,np.std])

In [0]:
df_pan2_patientMSTD

mean_nni                    sdnn  ...        sd2 ratio_sd2_sd1          
               mean         std        mean  ...        std          mean       std
Patient                                      ...                                   
2        693.637591  129.355436   21.063893  ...  34.873449      3.707411  1.496406
77       667.779902   74.662955   17.863944  ...  22.232340      4.100878  1.459307
107      765.697746   29.943443   26.229901  ...  13.630844      4.261284  0.784969
143      607.178423   49.179315   74.346817  ...  20.738122      3.716811  0.527525
148      641.222128   25.423834    8.837316  ...  10.195861      6.904118  2.745870
...             ...         ...         ...  ...        ...           ...       ...
13112    628.402507   86.198260   38.071883  ...  47.667296      4.048540  2.191076
13145    725.832916  151.670851   30.274507  ...  48.320550      4.351166  1.820279
13182    377.035714  533.209021  142.662978  ...        NaN           NaN       NaN
13400    217.140172  228.345734   25.314719  ...        NaN           NaN       NaN
13407    881.499160  258.000810   66.239717  ...  72.965425      3.138273  1.282986

[458 rows x 54 columns]

In [0]:
df_pan2_patientMSTD.to_csv('./pan2_patientMSTD.csv')

In [0]:
df_swt2_patientMSTD.to_csv('./swt2_patientMSTD.csv')

In [0]:
df_xqrs2_patientMSTD.to_csv('./xqrs2_patientMSTD.csv')

In [0]:
df_test = pd.read_csv("xqrs_v2.csv")

In [0]:
df_test['crisis_frequency'] = df_test['label'].apply(lambda x: 1 if (x!=0) else 0 )
                                                                    

In [0]:
df_test

,Unnamed: 0,Set,Categorie Montage,Dossier Patient,Patient,Session,File N°,exam_duration,SWT_vs_XQRS,Pan_vs_XQRS,interval_index,interval_start_time,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label,crisis_frequency
0,0,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,0.0,0.0,NaN,NaN,NaN,5.0,26.315789,6.0,31.578947,NaN,NaN,179.687500,NaN,NaN,NaN,132.987013,95.108359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
1,1,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,1.0,10000.0,537.109375,56.359305,41.722014,4.0,21.052632,4.0,21.052632,41.722578,546.875000,189.453125,0.077680,0.104931,113.041243,141.566820,97.834395,1.313485e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2,2,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,2.0,20000.0,602.213542,51.251884,26.831113,4.0,22.222222,4.0,22.222222,27.702479,585.937500,125.000000,0.046001,0.085106,100.296032,108.936170,88.786127,8.031925e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
3,3,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,3.0,30000.0,564.453125,33.125314,21.511084,1.0,5.555556,2.0,11.111111,21.687087,548.828125,89.843750,0.038421,0.058686,106.617186,110.503597,94.814815,5.605881e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
4,4,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1329,0.651163,0.476072,4.0,40000.0,NaN,NaN,NaN,2.0,11.111111,2.0,11.111111,NaN,NaN,125.000000,NaN,NaN,NaN,139.636364,108.169014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308940,308940,dev,02_tcp_le,46,4671,s005_2010_09_20,t001,1220,1.000000,1.000000,117.0,1170000.0,525.473684,1.982379,0.916246,0.0,0.000000,0.0,0.000000,0.942809,524.000000,4.000000,0.001794,0.003773,114.184229,114.503817,113.636364,4.184389e-01,11.091962,1.062998,8.230032,10.434598,8.199485,2.436866,388.251432,0.433293,1.443710,11.837678,8.199485,0.0,0
308941,308941,dev,02_tcp_le,46,4671,s005_2010_09_20,t001,1220,1.000000,1.000000,118.0,1180000.0,524.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,524.000000,0.000000,0.000000,0.000000,114.503817,114.503817,114.503817,1.421085e-14,9.152304,1.224314,24.660710,7.475457,8.156070,2.369910,356.595885,0.354601,1.340153,10.930383,8.156070,0.0,0
308942,308942,dev,02_tcp_le,46,4671,s005_2010_09_20,t001,1220,1.000000,1.000000,119.0,1190000.0,528.000000,2.981424,1.885618,0.0,0.000000,0.0,0.000000,1.885618,528.000000,8.000000,0.003571,0.005647,113.639796,114.503817,112.781955,6.245934e-01,4.562690,0.861958,8.318826,5.293405,9.782922,2.304168,434.296810,0.187326,1.134461,11.098341,9.782922,0.0,0
308943,308943,dev,02_tcp_le,46,4671,s005_2010_09_20,t001,1220,1.000000,1.000000,120.0,1200000.0,555.111111,11.023445,3.049760,0.0,0.000000,0.0,0.000000,3.360672,560.000000,28.000000,0.006054,0.019858,108.127022,111.111111,105.633803,2.101548e+00,9.082972,1.413336,28.909886,6.426618,6.653388,2.054814,182.797907,0.211488,1.032347,6.868603,6.653388,0.0,0


In [0]:
df_frequency = df_test.groupby(["Patient"]).crisis_frequency.mean()

In [0]:
df_finalxqrs = pd.merge(df_xqrs2_patient, df_frequency, on='Patient')



In [0]:
df_finalxqrs

,Patient,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,crisis_frequency
0,2,689.094620,20.214871,13.630261,0.664495,4.748278,1.022801,7.080544,14.052637,687.756272,60.274510,0.020464,0.029706,inf,inf,inf,2.711351,543.083750,150.288790,298.929860,4.932077,4.037827,3.509169,686.517017,inf,11.423463,39.825812,4.037827,0.231270
1,17,707.162115,13.878511,6.989932,0.096774,0.762609,0.266129,2.368312,7.589630,711.121951,38.322581,0.010047,0.018255,87.069804,90.450279,85.426638,1.618941,122.302838,51.284593,114.507835,4.771547,6.103743,3.338721,1335.665529,inf,6.784225,42.215218,6.103743,0.008065
2,21,696.028690,22.806232,13.164502,0.250000,1.613923,1.528846,10.459025,13.652229,696.784314,69.485437,0.019812,0.033237,87.052341,93.290598,83.619691,2.895940,382.529254,97.952686,211.854759,4.214278,4.303563,3.822175,763.514127,inf,10.523340,43.435157,4.303563,0.000000
3,32,993.491279,67.479057,46.719408,2.146341,21.301321,3.634146,35.765443,48.976662,996.224490,185.178862,0.050820,0.071014,62.081716,70.224919,57.023263,4.638969,3538.150468,464.979305,1832.277570,8.364500,3.676932,4.830773,2046.067838,inf,35.572919,130.733513,3.676932,0.008130
4,54,1135.040404,14.443546,10.517722,0.061538,0.683761,0.546154,6.176768,11.282227,1136.692308,40.061538,0.009967,0.012792,52.979610,54.198363,52.266691,0.650681,264.579598,14.727422,395.944448,12.751943,4.111332,3.427560,597.404172,0.863294,7.102295,30.564006,4.111332,0.007692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,13182,452.621128,107.964996,78.912937,2.727273,19.283804,3.636364,25.282022,81.000349,456.805556,257.469697,0.120777,0.161595,inf,inf,inf,16.555437,0.000000,0.000000,0.000000,NaN,NaN,-inf,NaN,inf,0.000000,0.000000,NaN,0.000000
599,13336,804.533667,30.412083,19.560937,0.514970,4.105575,2.688623,22.155210,20.294909,804.906627,84.165669,0.025245,0.038030,inf,inf,inf,2.799463,812.139345,183.612274,479.894534,8.484760,4.144689,4.104616,1110.554204,inf,14.950376,60.681075,4.144689,0.005917
600,13345,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,inf,inf,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
601,13400,571.832390,34.707421,19.708418,1.427273,7.461393,2.596970,13.631439,20.297845,576.575212,121.932859,0.036677,0.063196,inf,inf,inf,7.398015,763.762012,260.933791,443.197619,6.732947,5.803596,4.044991,1587.535866,inf,15.507868,71.137202,5.803596,0.023866


In [0]:
df_finalxqrs.to_csv('./finalxqrs.csv')

In [0]:
df_test1 = pd.read_csv("pan_v2.csv")

In [0]:
df_test1['crisis_frequency'] = df_test1['label'].apply(lambda x: 1 if (x!=0) else 0 )
                                                                    

In [0]:
df_frequency1 = df_test1.groupby(["Patient"]).crisis_frequency.mean()

In [0]:
df_finalpan = pd.merge(df_pan2_patient, df_frequency1, on='Patient')

In [0]:
df_finalpan

,Patient,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,crisis_frequency
0,2,693.637591,21.063893,14.649339,0.790850,5.113004,1.339869,8.263170,15.095678,693.344173,73.233996,0.022427,0.031475,inf,inf,inf,2.994399,739.394553,255.548878,375.921710,4.088026,3.707411,3.506260,608.078094,inf,12.503538,40.861404,3.707411,0.231270
1,77,667.779902,17.863944,10.429604,0.401914,2.341446,1.086124,6.243777,10.766434,671.122995,61.016260,0.016610,0.028194,inf,inf,inf,2.801075,316.117986,124.595894,205.396530,4.285516,4.100878,3.597228,654.876397,inf,8.839287,35.472837,4.100878,0.004525
2,107,765.697746,26.229901,15.133804,0.292857,2.466605,1.735714,13.767375,15.840552,767.568345,74.728571,0.020499,0.033969,78.589267,82.996540,75.310326,2.571145,717.951499,51.604062,408.929975,14.604600,4.261284,3.928822,847.015903,inf,11.637414,48.903056,4.261284,0.007143
3,143,607.178423,74.346817,42.761736,2.726562,17.357498,5.414062,34.279310,43.745314,600.404762,205.156250,0.071449,0.120989,101.002786,121.301150,86.464009,11.659417,4347.961957,1151.095863,1388.180139,3.995294,3.716811,4.753176,1748.789501,inf,31.360888,115.981644,3.716811,0.000000
4,148,641.222128,8.837316,4.295348,0.000000,0.000000,0.072464,0.483954,4.531387,641.289855,25.623188,0.006998,0.013706,93.743404,95.791329,92.073489,1.237667,99.749001,11.380713,122.388524,11.202918,6.904118,3.086725,745.541470,0.689254,3.593647,23.965928,6.904118,0.007194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,13112,628.402507,38.071883,25.307219,1.509494,9.646236,2.664557,16.977911,25.973838,630.754455,135.657511,0.040180,0.059390,inf,inf,inf,5.943055,2128.236082,679.643988,1516.454991,2.917690,4.048540,4.110577,1048.117694,inf,19.689407,68.127685,4.048540,0.003165
454,13145,725.832916,30.274507,17.721334,1.843762,7.898500,3.586086,14.735414,18.362591,728.817083,180.346492,0.028611,0.048212,inf,inf,inf,5.150411,708.522386,222.500009,305.881356,4.750801,4.351166,3.424203,777.091937,inf,10.628855,42.859950,4.351166,0.043629
455,13182,377.035714,142.662978,96.869231,3.625000,21.834734,4.375000,26.656162,98.362946,412.500000,372.413542,0.130442,0.189190,inf,inf,inf,15.246631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,NaN,NaN,NaN,0.477912
456,13400,217.140172,25.314719,11.171619,1.380328,4.391286,3.209836,9.940924,11.512024,211.167279,170.535552,0.044310,0.095387,inf,inf,inf,13.771949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,NaN,NaN,NaN,0.066667


In [0]:
df_test2 = pd.read_csv("swt_v2.csv")

In [0]:
df_test2['crisis_frequency'] = df_test2['label'].apply(lambda x: 1 if (x!=0) else 0 )

In [0]:
df_frequency2 = df_test2.groupby(["Patient"]).crisis_frequency.mean()

In [0]:
df_finalswt = pd.merge(df_swt2_patient, df_frequency2, on='Patient')

In [0]:
df_finalswt

,Patient,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,crisis_frequency
0,2,703.418330,21.294165,15.603290,0.758755,5.280153,1.081712,7.350101,15.961345,701.347639,67.988281,0.023299,0.031306,inf,inf,inf,2.927858,321.781198,145.517317,129.789111,3.456506,3.616530,-inf,624.547543,inf,12.741346,41.484435,3.616530,0.325714
1,17,708.739993,12.125843,6.053458,0.072581,0.628200,0.185484,1.404410,6.590495,712.227642,33.451613,0.008935,0.016158,86.892914,89.445302,85.382617,1.358504,107.565258,42.367878,110.092689,5.041854,6.356871,3.310263,1470.206199,inf,6.150033,41.528152,6.356871,0.008065
2,21,694.252123,23.229208,12.611954,0.203883,1.349977,1.553398,10.531141,13.125022,694.198020,69.941748,0.019429,0.035131,87.568136,93.756598,83.642315,3.156829,511.363211,116.837558,240.594468,4.370739,4.314849,3.798244,742.936751,inf,10.150937,42.139659,4.314849,0.000000
3,32,1002.052501,64.730073,42.678740,1.821138,18.476731,3.333333,33.160830,45.227626,1007.190476,181.544715,0.046489,0.067480,61.538852,70.764190,56.989283,4.466556,3747.486395,492.874096,1965.666602,8.576448,3.860206,4.795529,2106.971894,inf,34.358463,130.660467,3.860206,0.008130
4,54,1134.827086,14.980731,11.287326,0.084615,0.931624,0.569231,6.397436,12.004665,1137.092308,41.461538,0.010651,0.013315,52.995672,54.277899,52.260186,0.680034,264.405955,14.140293,389.046051,13.605638,4.107421,3.437264,592.108864,0.912792,7.313092,30.810414,4.107421,0.007692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,13182,123.703239,169.361969,129.281133,0.593750,5.320270,0.906250,8.228213,134.700520,118.491935,89.453125,0.192474,0.251381,inf,inf,inf,25.034867,0.000000,0.000000,0.000000,NaN,2.870940,5.747590,3635.056948,inf,110.620133,316.485915,2.870940,0.037037
600,13336,43.120098,44.927695,23.787397,0.171429,1.071429,0.371429,2.976190,23.877555,46.647059,18.057143,0.091095,0.171749,inf,inf,inf,21.673744,5884.582590,2220.757104,425.615577,1.731545,3.862348,5.001374,2942.673713,inf,45.007501,180.041792,3.862348,0.005917
601,13345,211.711084,34.060876,28.480137,0.193548,1.637717,0.387097,3.490488,29.042956,216.161290,23.822581,0.042269,0.049424,inf,inf,inf,4.124894,0.000000,0.000000,0.000000,NaN,2.128888,-inf,738.982774,inf,32.879710,69.077176,2.128888,0.037313
602,13400,589.902430,20.762331,10.805011,0.434911,2.593143,0.860947,5.302111,11.339801,589.519231,62.263610,0.019284,0.035251,inf,inf,inf,3.708613,593.000615,211.004546,391.532909,11.216968,6.769411,3.755724,1522.015529,inf,11.180869,56.567996,6.769411,0.023095


In [0]:
df_finalpan.to_csv('./finalpan.csv')

In [0]:
df_finalswt.to_csv('./finalswt.csv')

In [0]:
df_finalswtMSTD = pd.merge(df_swt2_patientMSTD, df_frequency2, on='Patient')

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [0]:
df_finalpanMSTD = pd.merge(df_pan2_patientMSTD, df_frequency1, on='Patient')

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [0]:
df_finalxqrsMSTD = pd.merge(df_xqrs2_patientMSTD, df_frequency, on='Patient')

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [0]:
df_finalxqrsMSTD

,"(mean_nni, mean)","(mean_nni, std)","(sdnn, mean)","(sdnn, std)","(sdsd, mean)","(sdsd, std)","(nni_50, mean)","(nni_50, std)","(pnni_50, mean)","(pnni_50, std)","(nni_20, mean)","(nni_20, std)","(pnni_20, mean)","(pnni_20, std)","(rmssd, mean)","(rmssd, std)","(median_nni, mean)","(median_nni, std)","(range_nni, mean)","(range_nni, std)","(cvsd, mean)","(cvsd, std)","(cvnni, mean)","(cvnni, std)","(mean_hr, mean)","(mean_hr, std)","(max_hr, mean)","(max_hr, std)","(min_hr, mean)","(min_hr, std)","(std_hr, mean)","(std_hr, std)","(lf, mean)","(lf, std)","(hf, mean)","(hf, std)","(vlf, mean)","(vlf, std)","(lf_hf_ratio, mean)","(lf_hf_ratio, std)","(csi, mean)","(csi, std)","(cvi, mean)","(cvi, std)","(Modified_csi, mean)","(Modified_csi, std)","(sampen, mean)","(sampen, std)","(sd1, mean)","(sd1, std)","(sd2, mean)","(sd2, std)","(ratio_sd2_sd1, mean)","(ratio_sd2_sd1, std)",crisis_frequency
Patient,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,689.094620,126.893429,20.214871,29.505921,13.630261,18.604579,0.664495,1.143957,4.748278,8.527089,1.022801,1.566438,7.080544,10.927144,14.052637,19.054596,687.756272,126.279885,60.274510,82.377479,0.020464,0.027611,0.029706,0.043529,inf,NaN,inf,NaN,inf,NaN,2.711351,4.273921,543.083750,671.843879,150.288790,108.692908,298.929860,453.096332,4.932077,4.055453,4.037827,1.928270,3.509169,0.895869,686.517017,821.656521,inf,NaN,11.423463,7.592627,39.825812,29.913307,4.037827,1.928270,0.231270
17,707.162115,152.821117,13.878511,38.998454,6.989932,12.421571,0.096774,0.532204,0.762609,4.817040,0.266129,0.797444,2.368312,7.875172,7.589630,15.774213,711.121951,159.930958,38.322581,94.395944,0.010047,0.016043,0.018255,0.037434,87.069804,9.786990,90.450279,13.577309,85.426638,10.087803,1.618941,3.394184,122.302838,84.219056,51.284593,66.988302,114.507835,54.300540,4.771547,3.807298,6.103743,2.481816,3.338721,0.575896,1335.665529,3878.966014,inf,NaN,6.784225,7.721040,42.215218,79.647088,6.103743,2.481816,0.008065
21,696.028690,86.124199,22.806232,12.152331,13.164502,7.477505,0.250000,0.634754,1.613923,4.006014,1.528846,1.214450,10.459025,8.011506,13.652229,7.564927,696.784314,86.735815,69.485437,45.405982,0.019812,0.011317,0.033237,0.018287,87.052341,6.344283,93.290598,12.156865,83.619691,6.001852,2.895940,1.846955,382.529254,176.737292,97.952686,55.309001,211.854759,158.346302,4.214278,1.041988,4.303563,1.084055,3.822175,0.240074,763.514127,335.975402,inf,NaN,10.523340,3.737200,43.435157,13.650543,4.303563,1.084055,0.000000
32,993.491279,143.036646,67.479057,44.442820,46.719408,24.761886,2.146341,1.496898,21.301321,14.736327,3.634146,1.695067,35.765443,15.785104,48.976662,25.405881,996.224490,150.825736,185.178862,114.874346,0.050820,0.028110,0.071014,0.050939,62.081716,8.840301,70.224919,13.295390,57.023263,7.117771,4.638969,4.113357,3538.150468,1796.524085,464.979305,299.630733,1832.277570,1082.629033,8.364500,2.380916,3.676932,0.983092,4.830773,0.241424,2046.067838,1238.109027,inf,NaN,35.572919,9.171042,130.733513,48.349234,3.676932,0.983092,0.008130
54,1135.040404,49.731912,14.443546,17.751358,10.517722,10.171090,0.061538,0.298675,0.683761,3.318614,0.546154,0.817555,6.176768,9.203812,11.282227,11.189369,1136.692308,49.425670,40.061538,41.185486,0.009967,0.009547,0.012792,0.015169,52.979610,2.347810,54.198363,3.027972,52.266691,2.318820,0.650681,0.753168,264.579598,503.283871,14.727422,8.656657,395.944448,833.030889,12.751943,11.718463,4.111332,1.318584,3.427560,0.388816,597.404172,882.000329,0.863294,0.330927,7.102295,3.590909,30.564006,24.534834,4.111332,1.318584,0.007692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13182,452.621128,347.312971,107.964996,63.248617,78.912937,44.733071,2.727273,2.572583,19.283804,17.643976,3.636364,3.294624,25.282022,22.140573,81.000349,45.223624,456.805556,354.740349,257.469

In [0]:
df_finalswtMSTD.to_csv('./finalswtMSTD.csv')

In [0]:
df_finalpanMSTD.to_csv('./finalpanMSTD.csv')

In [0]:
df_finalxqrsMSTD.to_csv('./finalxqrsMSTD.csv')